In [1]:
import pandas as pd
import re
import openpyxl

In [2]:
tar_date = "2021-08-31"
bef_date = "2021-07-31"
pref = "D:/Github/Work/Tableau/Personnel"

In [3]:
hr = pd.read_excel(f"D:/Github/Work/Tableau/직원 명단_조직 기준_{tar_date.replace('-', '')[2:]}.xlsx")

# 인사 현황 데이터 전처리

# 건설본부만 뽑기

In [5]:
# 본사, 지사, 현장 설정
hr[["직급", "직무", "하위그룹"]] = hr[["직급", "직무", "하위그룹"]].astype("str")
hr["코스트센터 분류"] = hr.apply(lambda x:"지사" if ("사업소" in x["조직명"]) or ("베트남지사" in x["조직명"]) else ("본사" if re.search("^\d", x["코스트센터"]) else "현장"), axis=1)
hr.loc[hr["코스트센터 분류"] == "현장", "본부명(조직단위기준)"] = "현장"
hr["코스트센터 본부명"] = hr["본부명(조직단위기준)"]
# hr["코스트센터 팀명"] = hr["원소속명"]
hr["코스트센터 팀명"] = hr["조직명"]

hq_const = hr[hr["본부명(조직단위기준)"] == "건설본부"]
hq_const = hq_const.drop(["코스트센터 분류", "코스트센터 본부명", "코스트센터 팀명"], axis=1)
# hq_const.to_excel(f"{pref}/직원 명단_조직 기준_{tar_date.replace('-', '')[2:]}_건설본부만.xlsx", encoding="euc-kr", index=False)

hq_no_const = hr[hr["본부명(조직단위기준)"] != "건설본부"]
# hq_no_const.to_excel(f"{pref}/직원 명단_조직 기준_{tar_date.replace('-', '')[2:]}_건설본부 제외.xlsx", encoding="euc-kr", index=False)

## 건설기획팀에서 회신한 제외 인원 반영

In [6]:
# 210731
# num_list = [3142, 3743, 3782, 5023, 5801, 6029, 4374, 4487, 6319, 3249, 3301]
# 21083
num_list = [3142, 3782, 5023, 5856, 6029, 4374, 6319, 3249, 3301, 4487]
hr.loc[(hr["사번"].isin(num_list)), "제외여부"] = "본부인원에서만 제외"

## 제외 로직 적용

In [7]:
# "본부인원에서만 제외" 설정
hr["제외여부"] = hr.apply(lambda x:"본부인원에서만 제외" if (x["본부명(조직단위기준)"]=="경영본부") & ((x["직급"]=="인턴") | (x["직무"]=="비상계획") | (x["직무"]=="축구협") | (x["직무"]=="비서") | (x["직무"]=="미화") | (x["직무"]=="기사")) else x["제외여부"], axis=1)
hr["제외여부"] = hr.apply(lambda x:"본부인원에서만 제외" if ("대기" in x["조직명"] or x["직무"] == "휴직") else x["제외여부"], axis=1)
# hr["제외여부"] = hr.apply(lambda x:"본부인원에서만 제외" if x["직무"]=="휴직" else x["제외여부"], axis=1)

# "제외" 설정
hr.loc[(hr["하위그룹"].isin(["CS외주", "정보기술외주", "파견직"])) | (hr["재직여부"] == "퇴직") | (hr["원소속명"] == "비상근"), "제외여부"] = "제외"
# hr.loc[(hr["하위그룹"].isin(["CS외주", "정보기술외주", "촉탁직", "파견직"])) | (hr["재직여부"] == "퇴직"), "제외여부"] = "제외"

hr = hr.rename({"본부명(조직단위기준)":"본부명"}, axis=1)
hr = hr[["사번", "성명", "EMAIL", "본부명", "원소속코드", "원소속명", "조직코드", "조직명", "직급", "구분", "직종", "성별", "하위그룹", "재직여부", "퇴사일", "직무", "제외여부", "코스트센터", "코스트센터내역", "코스트센터 분류", "코스트센터 본부명", "코스트센터 팀명"]]

In [8]:
hr.to_csv(f"{pref}/직원 명단_조직 기준_{tar_date.replace('-', '')[2:]}_전처리 최종.csv", index=False, encoding="euc-kr")

# 직급별 인원 파일 만들기

In [13]:
wb = openpyxl.load_workbook(f"{pref}/{bef_date.replace('-', '')[2:]}/인원 추이_조직 기준_전처리_{bef_date.replace('-', '')[2:]}.xlsx")

In [14]:
hr.loc[hr["코스트센터 본부명"]=="회장직속", "코스트센터 본부명"] = "사장직속"
hr.loc[hr["코스트센터 본부명"]=="부회장직속", "코스트센터 본부명"] = "사장직속"

cls2cnt = hr[hr["제외여부"]!="제외"]["코스트센터 분류"].value_counts()
hq2cnt = hr[(hr["제외여부"]!="제외") & (hr["제외여부"]!="본부인원에서만 제외")]["코스트센터 본부명"].value_counts()
print(cls2cnt, end="\n\n")
print(hq2cnt)

현장    1084
본사     461
지사      90
Name: 코스트센터 분류, dtype: int64

현장        1084
개발영업본부     265
건설본부       149
경영본부        50
미래혁신본부      43
사장직속        21
Name: 코스트센터 본부명, dtype: int64


In [15]:
ws = wb["근무지별 추이(5년치)"]
i = 2
for idx, value in cls2cnt.items():
    ws[f"A{i}"] = tar_date
    if idx == "지사":
        ws[f"B{i}"] = f"{idx}외"
    else:
        ws[f"B{i}"] = idx
    ws[f"C{i}"] = value
    i += 1

ws = wb["본부별 월별 추이(2021년)"]
for idx, value in hq2cnt.items():
    ws.insert_rows(2)
    ws["A2"] = tar_date
    ws["B2"] = idx
    ws["C2"] = value

In [16]:
wb.save(f"{pref}/인원 추이_조직 기준_전처리_{tar_date.replace('-', '')[2:]}.xlsx")

# 발령 이력

In [10]:
pd.set_option("display.max_rows", None)

In [11]:
data = pd.read_csv("D:\Github\Work/발령이력_18년이후_송부.csv")

In [13]:
data[data["근무처"].str.contains("디지털플랫")]

,사번,성명,발령시작일,발령종료일,직급(위),조직코드,근무처,보직(직무)
1562,4328,최 영소,20210405,99991231,2급,50298106,디지털플랫폼팀,팀원
3876,5634,맹 은주,20210405,99991231,2급,50298106,디지털플랫폼팀,팀장
4881,5899,원 혁,20210405,99991231,2급,50298106,디지털플랫폼팀,팀원
5469,6032,박 현준,20210405,99991231,2급,50298106,디지털플랫폼팀,팀원
5558,6048,노 인주,20210405,99991231,3급,50298106,디지털플랫폼팀,팀원
5762,6090,김 기현,20210405,99991231,3급,50298106,디지털플랫폼팀,팀원
5880,6111,이 광호,20210405,99991231,3급,50298106,디지털플랫폼팀,팀원
5969,6128,이 동신,20210405,99991231,2급,50298106,디지털플랫폼팀,팀원
6637,6294,이 민우,20210405,99991231,3급,50298106,디지털플랫폼팀,팀원
6657,6297,김 석희,20210405,99991231,3급,50298106,디지털플랫폼팀,팀원


In [12]:
nums = data[data["근무처"].str.contains("디지털플랫")]["사번"].tolist()
data[data["사번"].isin(nums)]

,사번,성명,발령시작일,발령종료일,직급(위),조직코드,근무처,보직(직무)
1558,4328,최 영소,20181126,20181231,부장,50213077,광교IPARK,공무
1559,4328,최 영소,20190101,20190101,부장(갑),50213077,광교IPARK,공무
1560,4328,최 영소,20190102,20190930,부장(갑),50255400,디지털혁신팀,팀원
1561,4328,최 영소,20191001,20210404,2급,50255400,디지털혁신팀,팀원
1562,4328,최 영소,20210405,99991231,2급,50298106,디지털플랫폼팀,팀원
3869,5634,맹 은주,20180101,20180331,차장,50236862,BT프로젝트팀,팀원
3870,5634,맹 은주,20180401,20180507,차장,50245679,디지털혁신TFT,TFT팀장
3871,5634,맹 은주,20180508,20181125,차장,50245679,디지털혁신TFT,TFT팀장
3872,5634,맹 은주,20181126,20190609,차장,50255400,디지털혁신팀,팀장
3873,5634,맹 은주,20190610,20190930,차장,50255400,디지털혁신팀,팀장
